# Blackholes

Synthesizer has a collection of routines allowing us to also model the spectral energy distributions of black holes (i.e. AGN) and combine their emission with the emission of others galaxy components (i.e. stellar emission).

## Particle blackholes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from unyt import Msun, yr, Angstrom

from synthesizer.blackholes import UnifiedAGN
from synthesizer.particle import BlackHoles
from synthesizer.particle import Gas
from synthesizer import galaxy

# Set a random number seed to ensure consistent results
np.random.seed(42)

### Creating Particle Blackholes

Before generating some simple observational quantities from physical properties we first need to create a `BlackHoles` object. This object can be found in `synthesizer/particle/blackholes.py`. 

Lets create an instance of `BlackHoles` containing 4 fake black holes. To do so we can provide a number of optional keyword arguments but for now lets just provide their masses, metallicities, coordinates and accretion rates.

In [ ]:
# Make fake properties
n = 4
masses = 10**np.random.uniform(low=7, high=9, size=n)  # Msun
coordinates = np.random.normal(0, 1.5, (n, 3))  # cMpc
accretion_rates = np.full(n, 10)  # Msun / yr
metallicities = np.full(n, 0.01)  

# And get the black holes object
bh = BlackHoles(masses=masses, coordinates=coordinates, accretion_rates=accretion_rates, metallicities=metallicities)

For some emission models we require an inclination. This could, in principle be calculated from the simulation, but for now we can use an in-built method to generate random inclinations:

In [ ]:
bh.calculate_random_inclination()
print(bh.inclination)

#### Blackhole properties

On initialisation this automatically calculates a handful of properties if they are provided. While `particle.BlackHoles` need to be initialised with `masses` and `accretion_rates` for `parametric.BlackHoles` we have more flexibility (see below)


For example, if `masses` and `accretion_rates` are provided then `bolometric_luminosities` ($ L_{\rm \bullet, bol} = \epsilon_{r}\dot{M}_{\bullet}c^{2} $) are automatically calcualted. Note that the radiative efficency (`epsilon`) defaults to 0.1 but can be set to an array. 

In [ ]:
bh.bolometric_luminosities

In [ ]:
bh.eddington_ratio

In [ ]:
print(bh)

### Blackhole spectra and lines

Blackhole spectra (and lines) can be generated by combining the `BlackHoles` object with an emission model. An emission model translates the physical properties of the blackhole(s) (e.g. `mass`, `accretion_rate`, etc.) to an spectral energy distribution. These models are described in more detail in the associated workbook.

In [ ]:
# define the emission model
grid_dir = '/Users/sw376/Dropbox/Research/data/synthesizer/grids/dev'
emission_model = UnifiedAGN(disc_model='agnsed', grid_dir=grid_dir)

# generate the spectra
# at the moment this produces a list of dictionaries, but that's not what we want. Instead we want a n-dimensional Sed object for each component.
spectra = bh.get_spectra(emission_model)



We can now examine a single component, in this case the total:

In [ ]:
sed = spectra['total']
print(sed)
print(sed.luminosity)

In [ ]:
plt.plot(np.log10(sed.lam), np.log10(sed.luminosity))
plt.ylim(np.max(np.log10(sed.luminosity))-4, np.max(np.log10(sed.luminosity))+0.2)
plt.xlim([2,6])
plt.show()

#### Calculating black hole metallicity

If we want to calculate emission from the black hole we will need to know the metallicity of the emission regions surrounding the black hole. Above we could have passed an array of metallicities at instantiation but most of the time we will not know ahead of time what these values should be. Instead we can use the gas surrounding the black hole to calculate what this metallicity is. To do this we need to first create a `Galaxy` with both a `Gas` component and `BlackHoles`, again using fake data.

In [ ]:
# # Make fake gas properties
# ngas = 200
# ms = np.full(ngas, 10 ** 6.5)  # Msun
# pos = np.random.normal(0, 1.5, (ngas, 3))  # cMpc
# hsml = np.full(ngas, 0.75)  # cMpc
# metals = np.full(ngas, 0.01)

# # And make the gas object
# gas = Gas(masses=ms, metallicities=metals, coordinates=pos, smoothing_lengths=hsml)

# # And now create the galaxy 
# galaxy = galaxy(stars=None, gas=gas, black_holes=bh)

Now we have the galaxy we can use `galaxy.calculate_black_hole_metallicity()` to calculate the black holes' metallicity. This method will find all gas particles with smoothing lengths that intersect the black hole and calculate the mass weighted average of their metallicity. If a black hole does not find any gas neighbours then a default metallicity is set instead, by default this is solar metallicity (0.012) but can be overwritten by passing a new `default_metallicity` as shown below.

In [ ]:
# galaxy.calculate_black_hole_metallicity(default_metallicity=0.07)
# print("Z_BH =", galaxy.black_holes.metallicities)